In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import re

In [2]:
driver = webdriver.Chrome(executable_path=r"C:\Users\Mark\Anaconda3\selenium\webdriver\chrome\chromedriver.exe")

In [2]:
df_beer = pd.read_csv('./df_beer.csv', encoding = "latin1")

In [6]:
# scraper stopped running at this point
df_beer['url'][4:]

4         /beer/profile/199/29619/
5       /beer/profile/46317/16814/
6         /beer/profile/287/17112/
7        /beer/profile/1199/19960/
8        /beer/profile/1146/10672/
9           /beer/profile/112/412/
10          /beer/profile/64/6108/
11          /beer/profile/140/276/
12           /beer/profile/147/88/
13        /beer/profile/140/30420/
14        /beer/profile/147/84596/
15         /beer/profile/43153/92/
16         /beer/profile/140/1904/
17         /beer/profile/26/64228/
18        /beer/profile/220/74986/
19            /beer/profile/22/34/
20       /beer/profile/2681/34483/
21         /beer/profile/259/1708/
22          /beer/profile/207/645/
23           /beer/profile/35/104/
24        /beer/profile/220/49789/
25          /beer/profile/220/916/
26          /beer/profile/192/607/
27        /beer/profile/694/15881/
28          /beer/profile/252/731/
29        /beer/profile/1199/5441/
30       /beer/profile/1199/35036/
31          /beer/profile/64/9086/
32          /beer/pr

In [4]:
review_grabber = lambda x: re.search('overall:\s\d\.\d*(.*[\s\S]*\d characters)', x).group(1)

def review_cleaner(string):
    x = review_grabber(string)
    x = re.sub('\d* characters', '', x)
    x = re.sub('\n', ' ', x)
    x = re.sub(',', '', x)
    return x
    
def get_user_reviews_csv(url):
    counter = 0
    review_number = 1
    for i in range(0,101):
        driver.get(f'https://www.beeradvocate.com{url}?view=beer&sort=&start={i*25}')
        sleep(2)
        beer_page = driver.page_source
        beer_page_soup = BeautifulSoup(beer_page, 'lxml')
        
        reviews = beer_page_soup.find_all('div', {'id':'rating_fullview_content_2'})
        
        counter += 1
        print(f'{url} -- page {counter}')
        
        for count, review in enumerate(reviews):
            score = review.find('span', {'class': 'BAscore_norm'}).text
            breakdown = review.find('span', {'class': 'muted'}).text
            u_names = review.find('a', {'class':'username'}).text
            try:
                r_text = review_cleaner(reviews[count].text)
            except:
                r_text = "No Review"
                
            master_list = [str(review_number), url, score, breakdown, u_names, r_text]
            with open('./aws_user_reviews.csv', 'a+') as f:
                print(','.join(master_list), file=f)
            
            review_number += 1
    
    sleep(2)

In [5]:
df_beer['url'][0:2].apply(get_user_reviews_csv)

/beer/profile/1199/11757/ -- page 1
/beer/profile/1199/11757/ -- page 2
/beer/profile/64/2093/ -- page 1
/beer/profile/64/2093/ -- page 2


0    None
1    None
Name: url, dtype: object